In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import linear_model
from sklearn.model_selection import ShuffleSplit

In [4]:
#creating whole dataset
seventeen = pd.read_json("https://missingmigrants.iom.int/global-figures/2017/json")
sixteen = pd.read_json("https://missingmigrants.iom.int/global-figures/2016/json")
fifteen = pd.read_json("https://missingmigrants.iom.int/global-figures/2015/json")
fourteen = pd.read_json("https://missingmigrants.iom.int/global-figures/2014/json")

frames = [seventeen,sixteen,fifteen,fourteen]

df = pd.concat(frames)

In [5]:
#cleaning data
df = df.replace('', 0,regex=True)
df = df.replace('nan', 0, regex=True)
df.drop(['Information Source'])

df['Reported Date'] = pd.to_datetime(df['Reported Date'])
# the index will help us get back from number to category later
df['UNSD Geographical Grouping'], geoIndex = pd.Series(df['UNSD Geographical Grouping']).factorize()
df['Migrant Route'], migrantIndex = pd.Series(df['Migrant Route']).factorize()
df['Region of Incident'], regionIndex = pd.Series(df['Region of Incident']).factorize()

In [6]:
#separating data
fourteen = df[(df['Reported Date'] >= '2014-01-01') & (df['Reported Date'] < '2015-01-01')]
fifteen = df[(df['Reported Date'] >= '2015-01-01') & (df['Reported Date'] < '2016-01-01')]
sixteen = df[(df['Reported Date'] >= '2016-01-01') & (df['Reported Date'] < '2017-01-01')]
seventeen = df[(df['Reported Date'] >= '2017-01-01') & (df['Reported Date'] < '2018-01-01')]

# print(fourteen.shape)
print(fourteen.columns.values)

['Cause of death' 'Information Reliability' 'Information Source' 'Location'
 'Location Description' 'Migrant Route' 'Number Dead' 'Number Missing'
 'Number of Children' 'Number of Female' 'Number of Male'
 'Number of survivors' 'Region of Incident' 'Reported Date'
 'Total Dead and Missing' 'UNSD Geographical Grouping' 'URL' 'Web ID']


In [7]:
total_data = np.concatenate((np.array(fourteen),np.array(fifteen),np.array(sixteen),np.array(seventeen)))
np.random.shuffle(total_data)
y = total_data[:,11].astype(int) / (total_data[:,14].astype(int) + total_data[:,11].astype(int)) 
# y is what we predict -- total sruvivors over total survivors+missing and dead 
y = np.nan_to_num(y)
y = (100*y).astype(int)
x = np.column_stack((total_data[:,5], total_data[:,12],total_data[:,15]))
# x is the parts of the data we are using to predict Location, route, region, and Geographical Grouping
x[1]

/usr/lib/python3.6/site-packages/ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in true_divide
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/lib/python3.6/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in true_divide
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/lib/python3.6/site-packages/ipykernel_launcher.py:6: RuntimeWarning: overflow encountered in multiply
  


array([1, 8, 10], dtype=object)

In [8]:
train = 2000
x_tr = x[:train,:]
y_tr = y[:train]
x_ts = x[train:,:]
y_ts = y[train:]
from sklearn import linear_model


In [9]:
regr = linear_model.LinearRegression()
regr.fit(x_tr,y_tr)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [10]:
regr.coef_

array([ -4.70462670e+14,  -7.65846864e+15,   5.97509143e+15])

In [11]:
#training data
y_pred_tr = regr.predict(x_tr)
RSS = np.mean((y_pred_tr - y_tr) **2)/(np.std(y_tr)**2)
print("RSS: = {0:f}".format(RSS))
print("R^2: = {0:f}".format(1-RSS))
print("R^2= {0:f}".format(regr.score(x_tr,y_tr)))

RSS: = 0.993671
R^2: = 0.006329
R^2= 0.006329


In [12]:
#test data
y_pred_ts = regr.predict(x_ts)

RSS = np.mean((y_pred_ts - y_ts) **2)/(np.std(y_ts)**2)
print("RSS: = {0:f}".format(RSS))
print("R^2: = {0:f}".format(1-RSS))
for i in y_ts:
    print(y_ts[i],y_pred_ts[i])

RSS: = 451701064949588129380611129344.000000
R^2: = -451701064949588129380611129344.000000
0 2.59995936925e+15
0 2.59995936925e+15
0 2.59995936925e+15
0 2.59995936925e+15
0 2.59995936925e+15
0 2.59995936925e+15
0 2.59995936925e+15
0 -2.63268018953e+15
0 2.59995936925e+15
0 2.59995936925e+15
0 2.59995936925e+15
0 -3.37513206102e+15
0 2.59995936925e+15
0 -2.63268018953e+15
0 2.59995936925e+15
0 2.59995936925e+15
0 2.59995936925e+15
0 2.59995936925e+15
0 2.59995936925e+15
0 2.59995936925e+15
0 2.59995936925e+15
0 2.59995936925e+15
0 2.59995936925e+15
0 2.59995936925e+15
0 2.59995936925e+15
0 2.59995936925e+15
0 2.59995936925e+15
0 2.59995936925e+15
0 -2.63268018953e+15
0 2.59995936925e+15
0 2.59995936925e+15
0 2.59995936925e+15
0 2.59995936925e+15
0 2.59995936925e+15
0 2.59995936925e+15
0 2.59995936925e+15
0 2.59995936925e+15
0 2.59995936925e+15
0 2.59995936925e+15
0 2.59995936925e+15
0 2.59995936925e+15
0 2.59995936925e+15
0 2.59995936925e+15
0 -3.37513206102e+15
0 2.59995936925e+15
0 -2

0 2.59995936925e+15
0 2.59995936925e+15
0 2.59995936925e+15
0 2.59995936925e+15
0 2.59995936925e+15
0 2.59995936925e+15
0 2.59995936925e+15
0 2.59995936925e+15
0 2.59995936925e+15
0 2.59995936925e+15
0 2.59995936925e+15
0 -3.37513206102e+15
0 2.59995936925e+15
0 2.59995936925e+15
0 2.59995936925e+15
0 2.59995936925e+15
0 2.59995936925e+15
0 2.59995936925e+15
0 2.59995936925e+15
0 2.59995936925e+15
0 2.59995936925e+15
0 2.59995936925e+15
0 2.59995936925e+15
0 2.59995936925e+15
0 2.59995936925e+15
0 2.59995936925e+15
0 -1.85255269678e+16
0 2.59995936925e+15
0 2.59995936925e+15
0 2.59995936925e+15
0 2.59995936925e+15
0 2.59995936925e+15
0 2.59995936925e+15
84 -3.57360552991e+15
0 2.59995936925e+15
0 2.59995936925e+15
0 2.59995936925e+15
0 2.59995936925e+15
26 -3.64508012587e+16
0 2.59995936925e+15
0 2.59995936925e+15
0 2.59995936925e+15
0 2.59995936925e+15
0 2.59995936925e+15
0 2.59995936925e+15
0 2.59995936925e+15
0 -3.37513206102e+15
0 2.59995936925e+15
0 2.59995936925e+15
0 2.599959369

In [21]:
def initTest(df):
    df['total people'] = pd.to_numeric(df['Number of survivors']) + pd.to_numeric(df['Total Dead and Missing'])
    y = np.array((pd.to_numeric(df['Number of survivors']) / df['total people']).fillna(0))
    y = np.nan_to_num(y)
    y[y == np.abs(np.inf)] = 0
    
    y = (100*y).astype(int)

    df['percent female'] = (pd.to_numeric(df['Number of Female'])/ df['total people']).fillna(0)
    df['percent male'] =  (pd.to_numeric(df['Number of Male'])/ df['total people']).fillna(0)
    df['percent kids'] =  (pd.to_numeric(df['Number of Children']) / df['total people']).fillna(0)

    x = np.array(df[['UNSD Geographical Grouping', 'Migrant Route', 'Region of Incident','percent female', 'percent kids', 'percent male']])
    x = np.nan_to_num(x)
    x[:,3] = (100*)
    

    rs = ShuffleSplit(n_splits=3, test_size=.25, random_state=0)

    for train_index, test_index in rs.split(x):
        regr = linear_model.LinearRegression()
        regr.fit(x[train_index],y[train_index])
        print("train: ", regr.score(x[train_index],y[train_index]) )
        print("test: ", regr.score(x[test_index],y[test_index]))
initTest(sixteen)

/usr/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/lib/python3.6/site-packages/ipykernel_launcher.py:7: RuntimeWarning: overflow encountered in multiply
  import sys
/usr/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/usr/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer]

train:  0.274448319168


ValueError: Input contains NaN, infinity or a value too large for dtype('float64').